# Web scraping the weekly Japanese software videogame sales

First I want to obtain a dataframe for the top 30 videogame sales using only one week, in order to test the code previous to the data minning.

In the block below I am going to import the libraries that I'm using:

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
import datetime
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

I'm going to use a week where there is at least one new release, as the row structure is different, as we will see. 

Below, using `requests` and `BeautifulSoup` I will obtain all the text from the first post.

I will print it in screen to see the structure.

In [2]:
url_tab = 'https://www.resetera.com/threads/media-create-sales-week-46-2018-nov-12-nov-18.82357/'

In [3]:
def obtain_post(url):
    # With request and with the BeautifulSoup class we obtain the code of the page of the URL.
    post = requests.get(url)
    soup = BeautifulSoup(post.text)
    # Divide the code in posts
    post_data=soup.findAll("div", {"class": "bbWrapper"})
    # Obtain the text inside the first post
    return post_data[0].text


tab_top30=obtain_post(url_tab)
print(tab_top30)

 
Media Create Sales: Week 46, 2018 (Nov 12 - Nov 18) 

01./00.  [NSW] Pokemon: Let's Go, Pikachu! / Let's Go, Eevee! # <RPG> (Pokemon Co.) {2018.11.16} (¥5.980) - 661.240 / NEW 
02./00.  [PS4] Fallout 76 # <RPG> (Bethesda Softworks) {2018.11.15} (¥7.980) - 73.489 / NEW 
03./03. [NSW] Super Mario Party <ETC> (Nintendo) {2018.10.05} (¥5.980) - 20.942 / 345.627 (-10%)
04./02. [PS4] Call of Duty: Black Ops IIII <ACT> (Sony Interactive Entertainment) {2018.10.12} (¥7.900) - 17.159 / 460.890 (-33%)
05./00.  [PS4] Hitman 2 <ADV> (Warner Entertainment Japan) {2018.11.15} (¥7.800) - 10.162 / NEW 
06./01. [3DS] Luigi's Mansion <ACT> (Nintendo) {2018.11.08} (¥4.980) - 9.320 / 36.689 (-66%)
07./05. [NSW] Mario Kart 8 Deluxe <RCE> (Nintendo) {2017.04.28} (¥5.980) - 9.319 / 1.836.063 (+10%)
08./04. [PS4] Red Dead Redemption II # <ACT> (Take-Two Interactive Japan) {2018.10.26} (¥8.800) - 8.120 / 192.181 (-52%)
09./07. [NSW] Splatoon 2 # <ACT> (Nintendo) {2017.07.21} (¥5.980) - 7.992 / 2.698.886 (+18

I want to obtain the Famitsu Sales, as the Media Create no longer share the data. So I need to identify where the Famitsu numbers begin and divide the rows. 

In [4]:
def obtain_top30_text(post_text):
    tab_top=post_text.split("Famitsu Sales: Week")[1]
    tab_top=tab_top.replace(" # ","").split("\n")
    return tab_top[2:32]

clean_tab=obtain_top30_text(tab_top30)
clean_tab

["01./00.  [NSW] Pokemon: Let's Go, Pikachu! / Let's Go, Eevee!<RPG> (Pokemon Co.) {2018.11.16} (¥5.980) - 664.198 / NEW <40-60%> ",
 '02./00.  [PS4] Fallout 76<RPG> (Bethesda Softworks) {2018.11.15} (¥7.980) - 60.222 / NEW <60-80%> ',
 '03./02. [NSW] Super Mario Party <ETC> (Nintendo) {2018.10.05} (¥5.980) - 17.486 / 326.510 <80-100%> (-15%)',
 '04./03. [PS4] Call of Duty: Black Ops IIII <ACT> (Sony Interactive Entertainment) {2018.10.12} (¥7.900) - 13.868 / 442.995 <80-100%> (-32%)',
 "05./01. [3DS] Luigi's Mansion <ACT> (Nintendo) {2018.11.08} (¥4.980) - 10.716 / 40.121 <40-60%> (-64%)",
 '06./00.  [PS4] Hitman 2 <ADV> (Warner Entertainment Japan) {2018.11.15} (¥7.800) - 10.688 / NEW <20-40%> ',
 '07./05. [NSW] Mario Kart 8 Deluxe <RCE> (Nintendo) {2017.04.28} (¥5.980) - 9.948 / 1.849.467 <80-100%> (+14%)',
 '08./04. [PS4] Red Dead Redemption II<ACT> (Take-Two Interactive Japan) {2018.10.26} (¥8.800) - 7.642 / 189.465 <80-100%> (-48%)',
 '09./06. [NSW] Minecraft <ADV> (Microsoft Gam

Now I need to create the most important function, the one that takes a row of the table in string form as an input and returns a list of formated data. I need to identify for each datapoint a regular rexpression that allows me to collect the desired information.

The structure is different if it is a new release or not. Take a look for example *Fallout 76* and *Super Mario Party*. 

In [5]:
def mine_sw_info(temp):
    row = []
    # Postition and previous week position
    row.append(int(re.findall(r"[0-9].",temp)[0]))
    row.append(int(re.findall(r"[0-9].",temp)[1]))
    # Find all things between [ ] and obtain the first one to get the system
    row.append(re.findall(r"\[(.*?)\]",temp)[0])
    # Find all things between < > and obtain the penultimate one to get the genre
    row.append(re.findall(r"\<(.*?)\>",temp)[-2])
    # Find all things between { } and obtain the last one to get the day of release
    row.append(re.findall(r"\{(.*?)\}",temp)[-1])



    if re.findall(r"\((.*?)\)",temp)[-1].find("%") >= 0: # Not a new release
        # Company
        row.append(re.findall(r"\((.*?)\)",temp)[-3])
        # Price
        row.append(  int(re.findall(r"\((.*?)\)",temp)[-2].replace(".","").replace("¥","")))
        # Units this week
        row.append(  int(re.findall(r"\)\ \-(.*?)\/",temp)[-1].replace(".","")))
        # Total units
        row.append(  int(re.findall(r"\/(.*?)\<",temp)[-1].replace(".","")))
        # Percentage of sales
        row.append(re.findall(r"\<(.*?)\>",temp)[-1])
        # Change with respect to the last week in percentage
        row.append(  int(re.findall(r"\((.*?)\)",temp)[-1].replace("%","").replace(" ","")))
        # Add a boolean to indicate this is not a new release
        row.append(  False )
    else:
        # Company
        row.append(re.findall(r"\((.*?)\)",temp)[-2])
        # Price
        row.append(int(re.findall(r"\((.*?)\)",temp)[-1].replace(".","").replace("¥","")))
        # Units this week
        row.append(int(re.findall(r"\)\ \-(.*?)\/",temp)[-1].replace(".","")))
        # Total Units = Units this week
        row.append(int(re.findall(r"\)\ \-(.*?)\/",temp)[-1].replace(".","")))
        # Percentage of sales
        row.append(re.findall(r"\<(.*?)\>",temp)[-1])
        # Change with respect to the last week in percentage, so put 0 to indicate new
        row.append(0)
        # Add a boolean to indicate this is not a new release
        row.append(  True )
    
    # Lastly, the name of the game
    name=temp.split(f"[{row[2]}]")[-1].split(f"<{row[3]}>")[0]
    row.append(name.lstrip().rstrip())
    return row

With the previous function, now I can iterate through all the rows in the top 30 video game sales. I need to include a header to keep track of what is each column and to convert. Finally I convert it to a Pandas DataFrame and include the year, the number of week and the days range in the week.

In [6]:
def create_top30_list(table_in,year,week,days,date):
    # need to create the header and append the rows in the top 30 game list
    header = ["Position", "Position Last Week","System","Genre","Launch Date","Company","Price","Sales (this week)",
              "LTD Sales","Range Sales","Change from last week","Launch this week","Game Title"]
    table_out=[header]
    # For each row in the top 30 game list, it appends the formated info in the list
    for row in table_in:
        table_out.append(mine_sw_info(row))
    
    # Converting the list to a Pandas DataFrame
    df = pd.DataFrame(table_out[1:],columns=table_out[0])
    # Adds in this dataframe a column with the year, the week number,
    # and the range of days in the week
    df["Year"]=year
    df["Week"]=week
    df["Days"]=days  
    df["Date"]=date
    return df

Now I'm going to check that everything works as it should...

In [7]:
df2=create_top30_list(clean_tab,2018,46,"nov 12 / nov 18","12/11/2018")
df2.head()

,Position,Position Last Week,System,Genre,Launch Date,Company,Price,Sales (this week),LTD Sales,Range Sales,Change from last week,Launch this week,Game Title,Year,Week,Days,Date
0,1,0,NSW,RPG,2018.11.16,Pokemon Co.,5980,664198,664198,40-60%,0,True,"Pokemon: Let's Go, Pikachu! / Let's Go, Eevee!",2018,46,nov 12 / nov 18,12/11/2018
1,2,0,PS4,RPG,2018.11.15,Bethesda Softworks,7980,60222,60222,60-80%,0,True,Fallout 76,2018,46,nov 12 / nov 18,12/11/2018
2,3,2,NSW,ETC,2018.10.05,Nintendo,5980,17486,326510,80-100%,-15,False,Super Mario Party,2018,46,nov 12 / nov 18,12/11/2018
3,4,3,PS4,ACT,2018.10.12,Sony Interactive Entertainment,7900,13868,442995,80-100%,-32,False,Call of Duty: Black Ops IIII,2018,46,nov 12 / nov 18,12/11/2018
4,5,1,3DS,ACT,2018.11.08,Nintendo,4980,10716,40121,40-60%,-64,False,Luigi's Mansion,2018,46,nov 12 / nov 18,12/11/2018


And it does!


![it works](https://wb4son.com/wpblog/wp-content/uploads/2016/09/Alive.gif)

### Getting all the data available to create the dataset

The next step is to use this function *urbi et orbi*. I mean, to all post with this info in the ResetEra forum. 

There is a post with all links to each weekly post, so let's web scrap it... In the block of code below I am going to retrieve all links in the post and store them in `links`.

In [8]:
url="https://www.resetera.com/threads/media-create-sales-archive-thread.4115/"
ustitles = requests.get(url)
soup = BeautifulSoup(ustitles.text)
links=soup.findAll("a", {"class": "link link--internal"}) 

Ok, so lets build the final loop to obtain the dataset.

In [9]:
# List with the URL of each post
links_clean=[]
# List with the year, week number and range of days in the week
links_data=[]
# Empty dataframe where I will store the final dataset
df_sw = pd.DataFrame()
# A list with the links where something failed and the script didn't work
error=[]
# Dictionary with the numerical values of the month
months={"jan":"1","feb":"2","mar":"3","apr":"4","may":"5","jun":"6","jul":"7","aug":"8","sep":"9","oct":"10","nov":"11","dec":"12"}

# Looping to mine the weekly data
for el in links[::-1]:
    # There are links where it is stored the yearly info instead of the weekly ones. 
    # We do not want these ones, so we use the following if statement to discard those.
    # They have "new-used" in the title and in their URL, so it is easy.
    if ( el.get('href').find("new-used") < 0):
        # Store the URL in a file
        links_clean.append(el.get('href'))
        time.sleep(1) # Just in case, to not to get an error due to do a lot of requests
        try:
            # Get the text table
            text_tab=obtain_top30_text(obtain_post(links_clean[-1]))
            # Obtain the year, week number and range of days in the week from the URL
            # and store them in links_data
            temp=links_clean[-1].split(".")[2].split("-")[-6:]
            temp_data=[int(temp[1]),int(temp[0]),temp[2]+" "+temp[3]+" / "+temp[4]+" "+temp[5],
                              temp[3]+"/"+months[temp[2]]+"/"+temp[1]]
            links_data.append(temp_data)
            # Append the weekly info dataframe in the final dataset
            df_sw=df_sw.append(create_top30_list(text_tab,temp_data[0],temp_data[1],temp_data[2],temp_data[3]))
        except:  # If there was an erron, store the URL in the error list
            error.append(links_clean[-1])

# Reseting the final dataframe index
df_sw=df_sw.reset_index()
# Convert Date column to datetime
df_sw["Date"]=pd.to_datetime(df_sw["Date"], infer_datetime_format=True)


In [10]:
df_sw.head()

,index,Position,Position Last Week,System,Genre,Launch Date,Company,Price,Sales (this week),LTD Sales,Range Sales,Change from last week,Launch this week,Game Title,Year,Week,Days,Date
0,0,1,1,NSW,TBL,2020.11.19,Konami,6300,107064,1617601,80-100%,23,False,"Momotaro Dentetsu: Showa, Heisei, Reiwa mo Tei...",2021,2,jan 11 / jan 17,2021-11-01
1,1,2,2,NSW,ETC,2020.03.20,Nintendo,5980,36009,6553618,80-100%,-20,False,Animal Crossing: New Horizons,2021,2,jan 11 / jan 17,2021-11-01
2,2,3,3,NSW,HOB,2019.10.18,Nintendo,7980,35397,2237398,80-100%,-20,False,Ring Fit Adventure,2021,2,jan 11 / jan 17,2021-11-01
3,3,4,4,NSW,RCE,2017.04.28,Nintendo,5980,26636,3601699,80-100%,-22,False,Mario Kart 8 Deluxe,2021,2,jan 11 / jan 17,2021-11-01
4,4,5,5,NSW,FTG,2018.12.07,Nintendo,7200,17099,4109327,80-100%,-29,False,Super Smash Bros. Ultimate,2021,2,jan 11 / jan 17,2021-11-01


In [11]:
df_sw.tail()

,index,Position,Position Last Week,System,Genre,Launch Date,Company,Price,Sales (this week),LTD Sales,Range Sales,Change from last week,Launch this week,Game Title,Year,Week,Days,Date
5035,25,26,23,NSW,FTG,2017.09.07,Bandai Namco Games,6800,2011,45219,80-100%,-33,False,Dragon Ball: Xenoverse 2 for Nintendo Switch,2017,42,oct 16 / oct 22,2017-10-16
5036,26,27,17,PS4,RPG,2017.09.28,Nihon Falcom,7800,1966,120050,80-100%,-55,False,The Legend of Heroes: Trails of Cold Steel III,2017,42,oct 16 / oct 22,2017-10-16
5037,27,28,25,PS4,RPG,2017.07.29,Square Enix,8980,1927,1321426,80-100%,-22,False,Dragon Quest XI: Echoes of an Elusive Age,2017,42,oct 16 / oct 22,2017-10-16
5038,28,29,28,PS4,ACT,2015.10.08,Take-Two Interactive Japan,4990,1907,304725,80-100%,-8,False,Grand Theft Auto V [New Price Edition],2017,42,oct 16 / oct 22,2017-10-16
5039,29,30,13,PS4,SLG,2017.10.14,Sony Interactive Entertainment,4900,1906,6604,80-100%,-59,False,No Heroes Allowed! VR |PlayStation VR|,2017,42,oct 16 / oct 22,2017-10-16


Only 2 of them returns an error, so we've got a very complete dataset using web scraping.

In [12]:
error

['https://www.resetera.com/threads/media-create-sales-week-32-2019-aug-05-aug-11.134991/',
 'https://www.resetera.com/threads/media-create-sales-week-19-2019-may-06-may-12.117093/']

And now I leave it to you to have fun with the dataset!

### A bit of fun with the database

Now I will play a little bit with the dataset. For example, let's see the popularity of genres by checking the sum of the titles in the top 30. It seems that the most popular ones are action, RPG and adventure games.

In [13]:
df_sw["Genre"].value_counts()

ACT    1534
RPG     881
ADV     812
SPT     403
ETC     357
FTG     303
RCE     219
SLG     205
HOB     148
TBL      72
EDU      49
PZL      40
STG      17
Name: Genre, dtype: int64

Now let's check the number of times a company made a game to be in the top 30. Nintendo seems to be dominating the top 30 charts in Japan, followed by Bandai Namco and Square Enix.

In [14]:
df_sw["Company"].value_counts()[:10]

Nintendo                          1896
Bandai Namco Games                 504
Square Enix                        316
Pokemon Co.                        287
Sony Interactive Entertainment     192
Capcom                             179
Konami                             155
Sega                               148
Microsoft Game Studios             144
Koei Tecmo                         141
Name: Company, dtype: int64

And now the times a company made a game to be the best selling video game in the week. As before the most succesful company is Nintendo. The second and the third places belongs to Sony and the Pokemon Company.

In [15]:
df_sw[df_sw["Position"] == 1]["Company"].value_counts()[:10]

Nintendo                          66
Pokemon Co.                       16
Sony Interactive Entertainment    16
Konami                            13
Bandai Namco Games                 9
Capcom                             8
Square Enix                        7
Sega                               5
Koei Tecmo                         4
Atlus                              4
Name: Company, dtype: int64

Checking the number of times a title made in the top 30, we see that there are 3 titles that, within the time range of the dataset, they appear in all top 30 weekly charts.

In [16]:
df_sw["Game Title"].value_counts()[:10]

Mario Kart 8 Deluxe                                                             168
The Legend of Zelda: Breath of the Wild                                         168
Splatoon 2                                                                      168
Super Mario Odyssey                                                             138
Minecraft                                                                       133
Super Mario Party                                                               118
Super Smash Bros. Ultimate                                                      109
New Super Mario Bros. U Deluxe <New Super Mario Bros. U \ New Super Luigi U>    103
Taiko no Tatsujin: Drum 'n' Fun!                                                101
Pokemon Ultra Sun / Ultra Moon                                                   82
Name: Game Title, dtype: int64

Now let's see the number of times a game made in the top of the list. Animal Crossing is the winner!

In [17]:
df_sw[df_sw["Position"] == 1]["Game Title"].value_counts()[:10]

Animal Crossing: New Horizons                         20
Pokemon Sword / Shield                                10
Momotaro Dentetsu: Showa, Heisei, Reiwa mo Teiban!     9
Super Smash Bros. Ultimate                             7
Super Mario Maker 2                                    6
Monster Hunter: World                                  5
Splatoon 2                                             4
Ring Fit Adventure                                     4
Kirby Star Allies                                      3
Marvel's Spider-Man                                    3
Name: Game Title, dtype: int64

Finally, I will put a list with the 10 most sucessful launches within the range of time of the dataset:

In [18]:
df_sw[df_sw["Launch this week"]].sort_values(by=['Sales (this week)'],ascending=False).head(10)

,index,Position,Position Last Week,System,Genre,Launch Date,Company,Price,Sales (this week),LTD Sales,Range Sales,Change from last week,Launch this week,Game Title,Year,Week,Days,Date
1290,0,1,0,NSW,ETC,2020.03.20,Nintendo,5980,1880626,1880626,80-100%,0,True,Animal Crossing: New Horizons,2020,12,mar 16 / mar 22,2020-03-16
1830,0,1,0,NSW,RPG,2019.11.15,Pokemon Co.,5980,1364544,1364544,60-80%,0,True,Pokemon Sword / Shield,2019,46,nov 11 / nov 17,2019-11-11
4590,0,1,0,PS4,ACT,2018.01.26,Capcom,8980,1350412,1350412,80-100%,0,True,Monster Hunter: World,2018,4,jan 22 / jan 28,2018-01-22
3240,0,1,0,NSW,FTG,2018.12.07,Nintendo,7200,1238358,1238358,60-80%,0,True,Super Smash Bros. Ultimate,2018,49,dec 03 / dec 09,2018-03-12
4890,0,1,0,3DS,RPG,2017.11.17,Pokemon Co.,4980,1199814,1199814,60-80%,0,True,Pokemon Ultra Sun / Ultra Moon,2017,46,nov 13 / nov 19,2017-11-13
1200,0,1,0,PS4,RPG,2020.04.10,Square Enix,8980,702853,702853,80-100%,0,True,Final Fantasy VII Remake,2020,15,apr 06 / apr 12,2020-06-04
3330,0,1,0,NSW,RPG,2018.11.16,Pokemon Co.,5980,664198,664198,40-60%,0,True,"Pokemon: Let's Go, Pikachu! / Let's Go, Eevee!",2018,46,nov 12 / nov 18,2018-12-11
3030,0,1,0,PS4,RPG,2019.01.25,Square Enix,8800,640406,640406,60-80%,0,True,Kingdom Hearts III,2019,4,jan 21 / jan 27,2019-01-21
4980,0,1,0,NSW,ACT,2017.10.27,Nintendo,5980,511625,511625,80-100%,0,True,Super Mario Odyssey,2017,43,oct 23 / oct 29,2017-10-23
240,0,1,0,NSW,TBL,2020.11.19,Konami,6300,345697,345697,60-80%,0,True,"Momotaro Dentetsu: Showa, Heisei, Reiwa mo Tei...",2020,47,nov 16 / nov 22,2020-11-16


And now... it's your turn to play with the dataset!